In [31]:
import boto3
import botocore
import os
import pandas as pd
from io import StringIO
import itertools

In [60]:
import datetime
from collections import Counter

In [29]:


import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler

1.13.1


Using TensorFlow backend.


In [30]:
StandardScaler()

StandardScaler(copy=True, with_mean=True, with_std=True)

In [49]:
def make_s3_resource():
    s3 = boto3.resource('s3',
            # aws_access_key_id=os.getenv('S3_BLOG_UPLOAD_ACCESS_KEY'),
            # aws_secret_access_key=os.getenv('S3_BLOG_UPLOAD_SECRET'),
            region_name='us-east-1')
    return s3


def write_s3_file(bucket_name, s3_filename, content):
    assert isinstance(content, basestring)
    s3conn = make_s3_resource()
    try:
        s3conn.Object(bucket_name, s3_filename).put(
                Body=content)
        return True
    except botocore.exceptions.ClientError as e:
        return False


def read_s3_file(bucket_name, s3_filename):
    s3conn = make_s3_resource()
    # try:
    return s3conn.Object(bucket_name, s3_filename).get()["Body"].read()
    # except botocore.exceptions.ClientError as e:


def s3_csv_to_df(bucket_name, s3_filename):
    blah = read_s3_file(bucket_name, s3_filename)
    foo = StringIO(blah.decode("utf-8"))
    return pd.read_csv(foo)

def big_s3_csv_to_df(bucket_name, s3_filename_prefix, suffixes):
    filenames = [s3_filename_prefix + suff
            for suff in suffixes]
    # return filenames
    parts = [read_s3_file(bucket_name, s3_filename) 
            for s3_filename in filenames ]
    blah = functools.reduce(lambda x, y: x+y, parts)
    foo = StringIO(blah.decode("utf-8"))
    return pd.read_csv(foo)

In [ ]:
import functools

In [53]:
blah = read_s3_file('my-sagemaker-blah',
         'aviation/small.csv')
# df = s3_csv_to_df('my-sagemaker-blah', 'aviation/small.csv')


suffixes = [
    '.part1' + letter
     for letter in ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
]

%time df = big_s3_csv_to_df('my-sagemaker-blah', 'aviation/sorted_train.csv', suffixes)
# 'sorted_train.csv.part1a'
# foo = StringIO(blah.decode("utf-8"))
# print(df.shape)

CPU times: user 28.4 s, sys: 9.24 s, total: 37.7 s
Wall time: 54.4 s


In [54]:
df.shape

(4867421, 28)

In [ ]:
# Third  model train, 
%time out, model = runner()

Start make_data 2019-05-17 01:13:03 EST
Start building training set 2019-05-17 01:13:03 EST
Start building testing set 2019-05-17 01:13:07 EST
metadata {'output': {'shapes': {'x_train': (446110, 256, 1), 'y_train': (446110, 4), 'x_test': (551326, 256, 1), 'y_test': (551326, 4), 'y_train_original': (446110,), 'y_test_original': (551326,), 'traindf': (447652, 7), 'testdf': (552868, 7)}, "Counter(outdata['y_train_original'])": {'A': 234352, 'C': 180851, 'D': 23218, 'B': 7689}, "Counter(outdata['y_test_original'])": {'C': 183718, 'A': 325198, 'D': 27039, 'B': 15371}}, 'input': {'kwargs': {'crews': {'training': [1], 'test': [2]}, 'percent_of_data': 1, 'sequence_window': 256, 'feature_cols': ['r']}}, 'data_ts': '2019-05-17 01:13:12 EST'}
Start bake_model 2019-05-17 01:13:12 EST
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train... 2019-05-17 01:13:13 E

In [ ]:
# Start bake_model 2019-05-17 01:13:12 UTC 
# Train on 446110 samples, validate on 551326 samples
epochs = [6575, 6553, 6531, 6498, 6492, 6483, 6486]
# 
# In [28]: start + datetime.timedelta(seconds=sum(epochs))                                                                                    
# Out[28]: datetime.datetime(2019, 5, 17, 13, 53, 30, tzinfo=<UTC>)
# ==> so Epoch 8 started around ^^ . 
# So 8 should finish around 
# In [33]: start + datetime.timedelta(seconds=sum(epochs)) + datetime.timedelta(seconds=round(sum(epochs)/7))                                 
# Out[33]: datetime.datetime(2019, 5, 17, 15, 42, 7, tzinfo=<UTC>)
# 
# and that indeed does happen.

'''
Epoch 1/10
 - 6575s - loss: 0.4677 - acc: 0.7747 - val_loss: 0.5016 - val_acc: 0.7502             
Epoch 2/10
 - 6553s - loss: 0.4582 - acc: 0.7668 - val_loss: 0.4974 - val_acc: 0.7946
Epoch 3/10
 - 6531s - loss: 0.4564 - acc: 0.7688 - val_loss: 0.4944 - val_acc: 0.7949
Epoch 4/10
 - 6498s - loss: 0.4492 - acc: 0.7788 - val_loss: 0.4997 - val_acc: 0.7677
Epoch 5/10
 - 6492s - loss: 0.4048 - acc: 0.8029 - val_loss: 0.4977 - val_acc: 0.7669
Epoch 6/10
 - 6483s - loss: 0.4045 - acc: 0.7974 - val_loss: 0.4971 - val_acc: 0.7667
Epoch 7/10
 - 6486s - loss: 0.3763 - acc: 0.8342 - val_loss: 0.4924 - val_acc: 0.7945
Epoch 8/10'''

In [56]:
# Okay so since there are 9 crews in this data , and indeed I assume the test data 
#     crews are the same people. Anyway, not too important for now.
#     But I can split the data into 6 for train and 3 for testing.
#
# - So as a preliminary simple model, I want to just use the `r` , the respiration data.
# - And I suppose it doesn't really matter all too much if both crew member data
#      are intermingled, but I think I will split that away for now.

encode_class = np.vectorize(lambda x: {'A': 0,
                                      'B': 1,
                                      'C': 2,
                                      'D': 3}.get(x))

simple_scaler = lambda x, a: x*a 

def timestamp():
    return datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S EST')

def do_standard_scaling(df, cols, scalar_dict=None):
    if scalar_dict is None:    
        scalar_dict = {col: StandardScaler().fit(df[[col]]) for col in cols}
        
    for col in cols:
        df[col + '_scaled'] = np.resize(
            scalar_dict[col].transform(df[[col]]),
            (df.shape[0],))
    
    return scalar_dict, df


def make_data(df, crews={'training': [1],
                        'test': [2]},
              sequence_window=256, percent_of_data=100,
             feature_cols={'r': 'standard_scaler'}):

    # current sorted as ['crew', 'experiment', 'time']
    [0, 1] # each seat
    ['CA', 'DA', 'SS'] # experiment
    
    sort_cols = ['crew', 'seat', 'experiment', 'time']
    target_col = 'event'
    
    what_cols = sort_cols + list(feature_cols) + [target_col]

    # Training
    traindf = df[df.crew.isin(crews['training'])][what_cols].sort_values(
        by=sort_cols).copy()
    
    scalar_dict, _ = do_standard_scaling(traindf, ['r'])
    
    print('Start building training set', timestamp())
    x_train, y_train = get_windows(traindf, ['r_scaled', 'event'],
                                   sequence_window,
                                  percent_of_data=percent_of_data)
    
    # Testing
    testdf = df[df.crew.isin(crews['test'])][what_cols].sort_values(
        by=sort_cols).copy()

    _, _ = do_standard_scaling(testdf, ['r'], scalar_dict)
    
    
    print('Start building testing set', timestamp())
    x_test, y_test = get_windows(testdf, ['r_scaled', 'event'],
                                 sequence_window,
                                 percent_of_data=percent_of_data)


    outdata = {
        "x_train": x_train,
        "y_train": reshape_y(encode_class(y_train), 4), # y_train,
        "x_test": x_test,
        "y_test": reshape_y(encode_class(y_test), 4), # y_test
        "y_train_original": y_train,
        "y_test_original": y_test,
        "traindf": traindf,
        "testdf": testdf,}
    metadata = {
        "metadata": {
            "output": {
                "shapes": {k: outdata[k].shape for k in list(outdata)},
                "Counter(outdata['y_train_original'])":
                dict(Counter(y_train)),
                "Counter(outdata['y_test_original'])":
                dict(Counter(y_test)),},
            "input": {"kwargs": {
                "crews": crews,
                "percent_of_data": percent_of_data,
                "sequence_window": sequence_window,
                "feature_cols": list(feature_cols)}},
            "data_ts": timestamp()
        }}
            
    return {**outdata, **metadata}
    
    
def runner():
    print('Start make_data', timestamp())
    outdata = make_data(df, crews={'training': [1],
                        'test': [2]},
              sequence_window=256, percent_of_data=1,
             feature_cols={'r': simple_scaler})
    
    validate_data(outdata)
    print('metadata', outdata['metadata'])
    print('Start bake_model', timestamp())
    model = bake_model(**outdata, epochs=10)
    return outdata, model

def validate_data(data):
    assert len(Counter(data['y_train_original'])) > 1
    assert len(Counter(data['y_test_original'])) > 1
  
    
def get_windows(df, cols, window_size, percent_of_data=100):
    
    whats_proportion_index = lambda x, y: round(x*y)
    
    X = []
    Y = []
    choices = (df.crew.unique().tolist(), [0, 1], ['CA', 'DA', 'SS'])
    for crew, seat, experiment in itertools.product(*choices):
        query = (df.crew == crew)&(df.seat == seat)&(df.experiment == experiment)
        thisdf = df[query][cols]
        X_i, Y_i = to_sequences(thisdf.values, window_size)
        X.append(X_i[:
                     whats_proportion_index(
                         X_i.shape[0],
                         percent_of_data)])
        Y.append(Y_i[:
                     whats_proportion_index(
                        Y_i.shape[0],
                        percent_of_data)])
        
    return np.concatenate(X), np.concatenate(Y)

# Borrowing parts of this func from 
# https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class10_lstm.ipynb
def to_sequences(obs, seq_size, incols=[0], outcols=[1]):
    x = []
    y = []

    for i in range(len(obs)-seq_size-1):
        #print(i)
        window = obs[i:(i+seq_size)][..., 0]
        after_window = obs[i+seq_size, 1] # FIXME :off by 1 error here?
        # window = [[x] for x in window]

        x.append(window)
        y.append(after_window)
        
    xarr = np.array(x)
    yarr = np.array(y)
    return (np.resize(xarr, xarr.shape + (1,)),
            yarr)

def bake_model(x_train, y_train, x_test, y_test, epochs=1, **kwargs):
    
    model = Sequential()
    model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2,
                   input_shape=(None, 1)))
    # model.add(Dense(32))

    # 4 because 'A', 'B', 'C', 'D'.
    model.add(Dense(4))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # initial_state ... for the LSTM , hmm


    monitor = EarlyStopping(monitor='val_loss',
                            min_delta=1e-3, patience=5, verbose=1, mode='auto')
    print('Train...', timestamp())

    # https://www.tensorflow.org/api_docs/python/tf/keras/models/Model#arguments_8
    # - hmm so fit() can take a generator sometimes.
    # - use_multiprocessing=True 
    model.fit(x_train, y_train, validation_data=(x_test, y_test),
              callbacks=[monitor], verbose=2, epochs=epochs)

    return model

def reshape_y(y, num_cols):

    # y = np.array([1,2,3,2,3,1],dtype=np.int32)

    # Convert y2 to dummy variables
    y2 = np.zeros((y.shape[0], num_cols), dtype=np.float32)
    y2[np.arange(y.shape[0]), y] = 1.0
    return y2